In [82]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

class Data():
    def __init__(self, batch_size=16, T=1000, num_train=600, tau=4):
        self.time = torch.arange(1, T + 1, dtype=torch.float32)
        self.x = torch.sin(0.01 * self.time) + torch.randn(T) * 0.2

data = Data()

df = pd.DataFrame({
    'time': data.time.numpy(),
    'x': data.x.numpy()
})

fig = px.scatter(df, x='time', y='x', title='Time Series Data')
fig.show()


In [93]:
tau = 4
num_samples = 1000

# Create lagged dataframe
lagged_data = {
    f'lag_{i}': df['x'].shift(i) for i in range(tau, 0, -1)
}
lagged_data['target'] = df['x']

lagged_df = pd.DataFrame(lagged_data)

# Drop rows with NaN (the first `tau` rows)
lagged_df = lagged_df.iloc[tau:num_samples]

print(lagged_df.head())
lagged_df


      lag_4     lag_3     lag_2     lag_1    target
4  0.092001 -0.395937  0.116697 -0.005946  0.044240
5 -0.395937  0.116697 -0.005946  0.044240  0.190818
6  0.116697 -0.005946  0.044240  0.190818  0.305289
7 -0.005946  0.044240  0.190818  0.305289  0.082765
8  0.044240  0.190818  0.305289  0.082765  0.076811


,lag_4,lag_3,lag_2,lag_1,target
4,0.092001,-0.395937,0.116697,-0.005946,0.044240
5,-0.395937,0.116697,-0.005946,0.044240,0.190818
6,0.116697,-0.005946,0.044240,0.190818,0.305289
7,-0.005946,0.044240,0.190818,0.305289,0.082765
8,0.044240,0.190818,0.305289,0.082765,0.076811
...,...,...,...,...,...
995,-0.689619,-0.425821,-0.885960,-0.565409,-0.550056
996,-0.425821,-0.885960,-0.565409,-0.550056,-0.609467
997,-0.885960,-0.565409,-0.550056,-0.609467,-0.425373
998,-0.565409,-0.550056,-0.609467,-0.425373,-0.369112


In [95]:
df_train = lagged_df[:600]
df_test = lagged_df[600:]
df_train, df_test

(        lag_4     lag_3     lag_2     lag_1    target
 4    0.092001 -0.395937  0.116697 -0.005946  0.044240
 5   -0.395937  0.116697 -0.005946  0.044240  0.190818
 6    0.116697 -0.005946  0.044240  0.190818  0.305289
 7   -0.005946  0.044240  0.190818  0.305289  0.082765
 8    0.044240  0.190818  0.305289  0.082765  0.076811
 ..        ...       ...       ...       ...       ...
 599 -0.030949  0.051371 -0.671007 -0.102631 -0.469045
 600  0.051371 -0.671007 -0.102631 -0.469045 -0.308310
 601 -0.671007 -0.102631 -0.469045 -0.308310 -0.333806
 602 -0.102631 -0.469045 -0.308310 -0.333806 -0.410774
 603 -0.469045 -0.308310 -0.333806 -0.410774 -0.069495
 
 [600 rows x 5 columns],
         lag_4     lag_3     lag_2     lag_1    target
 604 -0.308310 -0.333806 -0.410774 -0.069495 -0.493646
 605 -0.333806 -0.410774 -0.069495 -0.493646 -0.269365
 606 -0.410774 -0.069495 -0.493646 -0.269365 -0.250098
 607 -0.069495 -0.493646 -0.269365 -0.250098 -0.089664
 608 -0.493646 -0.269365 -0.250098 -0.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

y = df_train['target']
X = df_train.drop('target', axis=1)

model = LinearRegression().fit(X, y)

In [96]:
X_test = df_test.drop('target', axis=1)
y_test = df_test['target']

In [97]:
y_hat = model.predict(X_test)
mean_squared_error(y_hat, y_test)

0.05377797

In [104]:
import plotly.graph_objects as go

# Recover time index for predictions
# Note: lagged_df lost 'time', but you can rebuild it by shifting df['time'] by tau
# because lagged_df.iloc[tau:] corresponds to time[tau:]
time_for_preds = df['time'].iloc[lagged_df.index[600:]]

# Build figure
fig = go.Figure()

# Original time series
fig.add_trace(go.Scatter(
    x=df['time'],
    y=df['x'],
    mode='lines',
    name='Original Series'
))

# Optionally, also actual test target
fig.add_trace(go.Scatter(
    x=time_for_preds,
    y=y_test,
    mode='lines',
    name='Actual Test'
))

# Predictions
fig.add_trace(go.Scatter(
    x=time_for_preds,
    y=y_hat,
    mode='lines',
    name='Predictions'
))

fig.update_layout(
    title='Time Series with Predictions',
    xaxis_title='Time',
    yaxis_title='Value'
)

fig.show()


In [109]:
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# -------------------
# 1) Generate data
# -------------------
class Data():
    def __init__(self, batch_size=16, T=1000, num_train=600, tau=4):
        self.time = torch.arange(1, T + 1, dtype=torch.float32)
        self.x = torch.sin(0.01 * self.time) + torch.randn(T) * 0.2

data = Data()

df = pd.DataFrame({
    'time': data.time.numpy(),
    'x': data.x.numpy()
})

# -------------------
# 2) Build lag features
# -------------------
tau = 4
num_samples = 1000

lagged_data = {f'lag_{i}': df['x'].shift(i) for i in range(tau, 0, -1)}
lagged_data['target'] = df['x']

lagged_df = pd.DataFrame(lagged_data).iloc[tau:num_samples]

# -------------------
# 3) Train on first 600
# -------------------
df_train = lagged_df[:600]
X_train = df_train.drop('target', axis=1)
y_train = df_train['target']

model = LinearRegression().fit(X_train, y_train)

# -------------------
# 4) Predict next 32 steps starting from last known point
# -------------------
# Grab the last row of the last training example to start
last_known_inputs = lagged_df.drop('target', axis=1).iloc[599].values.reshape(1, -1)

future_preds = []
for _ in range(32):
    next_pred = model.predict(last_known_inputs)[0]
    future_preds.append(next_pred)
    
    # Shift window: drop oldest, add new prediction
    last_known_inputs = np.roll(last_known_inputs, -1)
    last_known_inputs[0, -1] = next_pred

# -------------------
# 5) Build future time indices
# -------------------
last_time = df['time'].iloc[599]  # last point of train
future_times = np.arange(last_time + 1, last_time + 1 + 32)

# -------------------
# 6) Plot original series + future predictions
# -------------------
fig = go.Figure()

# Original series
fig.add_trace(go.Scatter(
    x=df['time'],
    y=df['x'],
    mode='lines',
    name='Original Series'
))

# 32-step future forecast
fig.add_trace(go.Scatter(
    x=future_times,
    y=future_preds,
    mode='lines+markers',
    name='32-Step Forecast'
))

fig.update_layout(
    title='Forecasting 32 Steps Beyond Last Known Data (from t=600)',
    xaxis_title='Time',
    yaxis_title='Value'
)

fig.show()


c:\Users\rowek\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\rowek\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\rowek\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\rowek\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\rowek\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\rowek\